In [18]:
import ctypes
import os

CUDA_BASE = r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.0"
BIN_DIR   = fr"{CUDA_BASE}\bin"
NVVM_BIN  = fr"{CUDA_BASE}\nvvm\bin"

# Asegurá variables de entorno básicas para NVRTC/NVVM
os.environ["CUDA_PATH"] = CUDA_BASE
# (opcional pero ayuda a NVRTC a encontrar libdevice)
os.environ.setdefault("CUDA_CACHE_PATH", os.path.expanduser(r"~\AppData\Local\NVIDIA\ComputeCache"))

# Agregá las carpetas de DLLs al loader del proceso actual
for p in (BIN_DIR, NVVM_BIN):
    if os.path.exists(p):
        os.add_dll_directory(p)

# (Opcional) sanity check de DLLs clave
print("nvrtc:", os.path.exists(fr"{BIN_DIR}\x64\nvrtc64_130_0.dll"))
print("builtins:", os.path.exists(fr"{BIN_DIR}\x64\nvrtc-builtins64_130.dll"))
print("libdevice:", os.path.exists(fr"{CUDA_BASE}\nvvm\libdevice\libdevice.10.bc"))

# Cargá NVRTC explícitamente para ver si hay error de DLLs
ctypes.CDLL(fr"{BIN_DIR}\x64\nvrtc64_130_0.dll")

nvrtc: True
builtins: True
libdevice: True


<CDLL 'C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.0\bin\x64\nvrtc64_130_0.dll', handle 7ff889340000 at 0x1a29b814320>

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2
from itertools import combinations


# ajustá el nombre de la carpeta si no es exactamente v13.0
os.add_dll_directory(r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.0\bin")
os.environ["FLAMEGPU_LOG_LEVEL"] = "trace"  # or "debug"
import pyflamegpu as fg
print(fg.__version__)


print("Librerías cargadas correctamente")


2.0.0rc3+cuda130
Librerías cargadas correctamente


In [20]:
COUNTRY_DATA_PATH = "data/covid_19_country_daily_with_coords.csv"
GRAPH_DATA_PATH = "data/country_proximity_edges.csv"

country_df = pd.read_csv(COUNTRY_DATA_PATH, parse_dates=["ObservationDate"], dayfirst=False)
edge_df = pd.read_csv(GRAPH_DATA_PATH)

print(f"Registros país-fecha: {len(country_df):,}")
print(f"Aristas proximidad: {len(edge_df):,}")
country_df.head()


Registros país-fecha: 87,279
Aristas proximidad: 25,425


,Country/Region,ObservationDate,Confirmed,Deaths,Recovered,Latitud_promedio,Longitud_promedio
0,Azerbaijan,2020-02-28,1.0,0.0,0.0,40.14310,47.576900
1,Afghanistan,2021-01-01,51526.0,2191.0,41727.0,33.93911,67.709953
2,Afghanistan,2021-01-02,51526.0,2191.0,41727.0,33.93911,67.709953
3,Afghanistan,2021-01-03,51526.0,2191.0,41727.0,33.93911,67.709953
4,Afghanistan,2021-01-04,53011.0,2237.0,42530.0,33.93911,67.709953


In [21]:
# Lista ordenada de fechas y resumen final por país (última fecha disponible)
country_df = country_df.sort_values(["Country/Region", "ObservationDate"]).reset_index(drop=True)
unique_dates = country_df["ObservationDate"].sort_values().unique()
num_steps = len(unique_dates)
print(f"Fechas distintas en el dataset: {num_steps}")

latest_per_country = (
    country_df.sort_values("ObservationDate")
              .groupby("Country/Region", as_index=False)
              .tail(1)
              .reset_index(drop=True)
)
print(f"Países únicos: {len(latest_per_country):,}")
latest_per_country.head()


Fechas distintas en el dataset: 494
Países únicos: 226


,Country/Region,ObservationDate,Confirmed,Deaths,Recovered,Latitud_promedio,Longitud_promedio
0,North Ireland,2020-02-28,1.0,0.0,0.0,21.840266,-41.440047
1,Republic of Ireland,2020-03-08,21.0,0.0,0.0,53.142400,-7.692100
2,Vatican City,2020-03-09,1.0,0.0,0.0,41.902900,12.453400
3,China,2020-03-09,0.0,0.0,0.0,32.887645,111.785991
4,Palestine,2020-03-09,22.0,0.0,0.0,31.952200,35.233200


In [22]:
# Construimos los estados iniciales (S, E, I, R, D) por país
min_active = 0.1  # evita valores exactamente cero

latest_per_country["Active"] = (
    latest_per_country["Confirmed"]
    - latest_per_country["Recovered"]
    - latest_per_country["Deaths"]
).clip(lower=0.0)

latest_per_country["Population_proxy"] = (
    latest_per_country["Confirmed"]
    + latest_per_country["Recovered"]
    + latest_per_country["Deaths"]
) * 1.25

latest_per_country["Population_proxy"] = latest_per_country.apply(
    lambda row: max(row["Population_proxy"], row["Active"] + row["Recovered"] + row["Deaths"] + 1.0),
    axis=1
)

latest_per_country["I0"] = latest_per_country["Active"].clip(lower=min_active)
latest_per_country["R0"] = latest_per_country["Recovered"].clip(lower=0.0)
latest_per_country["D0"] = latest_per_country["Deaths"].clip(lower=0.0)
latest_per_country["S0"] = (
    latest_per_country["Population_proxy"]
    - latest_per_country["I0"]
    - latest_per_country["R0"]
    - latest_per_country["D0"]
).clip(lower=0.0)
latest_per_country["E0"] = 0.0

state_cols = ["Country/Region", "Latitud_promedio", "Longitud_promedio", "Population_proxy", "S0", "E0", "I0", "R0", "D0"]
state_df = latest_per_country[state_cols].rename(columns={"Population_proxy": "Population"}).reset_index(drop=True)
state_df.head()


,Country/Region,Latitud_promedio,Longitud_promedio,Population,S0,E0,I0,R0,D0
0,North Ireland,21.840266,-41.440047,2.00,1.00,0.0,1.0,0.0,0.0
1,Republic of Ireland,53.142400,-7.692100,26.25,5.25,0.0,21.0,0.0,0.0
2,Vatican City,41.902900,12.453400,2.00,1.00,0.0,1.0,0.0,0.0
3,China,32.887645,111.785991,1.00,0.90,0.0,0.1,0.0,0.0
4,Palestine,31.952200,35.233200,27.50,5.50,0.0,22.0,0.0,0.0


In [23]:
# Construimos matrices auxiliares: IDs, poblaciones y proximidades normalizadas
state_df = state_df.sort_values("Country/Region").reset_index(drop=True)
num_countries = len(state_df)

country_to_id = {name: idx for idx, name in enumerate(state_df["Country/Region"])}
id_to_country = {idx: name for name, idx in country_to_id.items()}

coords = state_df[["Latitud_promedio", "Longitud_promedio"]].to_numpy(dtype=float)
population = state_df["Population"].to_numpy(dtype=float)

# Distancia Haversine pairwise
RADIUS_EARTH_KM = 6371.0

def haversine(lat1, lon1, lat2, lon2):
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return RADIUS_EARTH_KM * c

distance_matrix = np.zeros((num_countries, num_countries), dtype=float)
for i in range(num_countries):
    for j in range(i + 1, num_countries):
        dist = haversine(coords[i, 0], coords[i, 1], coords[j, 0], coords[j, 1])
        distance_matrix[i, j] = distance_matrix[j, i] = dist

max_dist = distance_matrix.max()
proximity_matrix = 1.0 - (distance_matrix / max_dist)
np.fill_diagonal(proximity_matrix, 0.0)

print(f"Países (agentes): {num_countries}")
print(f"Distancia máxima: {max_dist:,.2f} km")
state_df.head()


Países (agentes): 226
Distancia máxima: 19,955.87 km


,Country/Region,Latitud_promedio,Longitud_promedio,Population,S0,E0,I0,R0,D0
0,Afghanistan,33.93911,67.709953,162863.75,92752.75,0.0,9931.0,57281.0,2899.0
1,Albania,41.15330,20.168300,329951.25,197654.25,0.0,633.0,129215.0,2449.0
2,Algeria,28.03390,1.659600,276668.75,148212.75,0.0,35577.0,89419.0,3460.0
3,Andorra,42.50630,1.521800,34045.00,20352.00,0.0,150.0,13416.0,127.0
4,Angola,-11.20270,17.873900,78228.75,44048.75,0.0,5777.0,27646.0,757.0


In [24]:
# Parámetros epidemiológicos razonables (puedes ajustarlos luego)
params = {
    "beta_local": 0.30,       # tasa de contagio interna
    "beta_travel": 0.05,      # influencia de viajes entre países
    "sigma": 1.0 / 5.0,       # 1/incubación ≈ 5 días
    "gamma": 1.0 / 7.0,       # 1/duración infecciosa ≈ 7 días
    "mortality": 0.01,        # 1% de los infectados activos mueren al recuperarse
    "dt": 1.0                 # paso temporal = 1 día
}
params


{'beta_local': 0.3,
 'beta_travel': 0.05,
 'sigma': 0.2,
 'gamma': 0.14285714285714285,
 'mortality': 0.01,
 'dt': 1.0}

In [25]:
# ======================================================
# Definicion del modelo pyflamegpu
# ======================================================
model = fg.ModelDescription("COVID_Country_Model")

# ----- agente -----
country_agent = model.newAgent("Country")
country_agent.newVariableUInt("id")
country_agent.newVariableFloat("population")
country_agent.newVariableFloat("S")
country_agent.newVariableFloat("E")
country_agent.newVariableFloat("I")
country_agent.newVariableFloat("R")
country_agent.newVariableFloat("D")
country_agent.newVariableFloat("lat")
country_agent.newVariableFloat("lon")

# ----- mensaje -----
message = model.newMessageBruteForce("InfectiousMsg")
message.newVariableUInt("id")
message.newVariableFloat("infectious")

output_infectious = r"""
    FLAMEGPU_AGENT_FUNCTION(output_infectious, flamegpu::MessageNone, flamegpu::MessageBruteForce) {
        const unsigned int id = FLAMEGPU->getVariable<unsigned int>("id");
        const float I = FLAMEGPU->getVariable<float>("I");
        FLAMEGPU->message_out.setVariable<unsigned int>("id", id);
        FLAMEGPU->message_out.setVariable<float>("infectious", I);
        return flamegpu::ALIVE;
    }
"""

update_state = r"""
    FLAMEGPU_AGENT_FUNCTION(update_state, flamegpu::MessageBruteForce, flamegpu::MessageNone) {
        const unsigned int id = FLAMEGPU->getVariable<unsigned int>("id");
        const unsigned int num_agents = FLAMEGPU->environment.getProperty<unsigned int>("num_agents");

        float S = FLAMEGPU->getVariable<float>("S");
        float E = FLAMEGPU->getVariable<float>("E");
        float I = FLAMEGPU->getVariable<float>("I");
        float R = FLAMEGPU->getVariable<float>("R");
        float D = FLAMEGPU->getVariable<float>("D");
        const float population = FLAMEGPU->getVariable<float>("population");

        const float beta_local = FLAMEGPU->environment.getProperty<float>("beta_local");
        const float beta_travel = FLAMEGPU->environment.getProperty<float>("beta_travel");
        const float sigma = FLAMEGPU->environment.getProperty<float>("sigma");
        const float gamma = FLAMEGPU->environment.getProperty<float>("gamma");
        const float mortality = FLAMEGPU->environment.getProperty<float>("mortality");
        const float dt = FLAMEGPU->environment.getProperty<float>("dt");

        const float inv_pop = 1.0f / (population + 1.0f);
        float local_force = beta_local * I * inv_pop;

        float travel_force = 0.0f;
        for (const auto &msg : FLAMEGPU->message_in) {
            const unsigned int other_id = msg.getVariable<unsigned int>("id");
            if (other_id == id) {
                continue;
            }
            const float prox = FLAMEGPU->environment.getProperty<float>("proximity_matrix", id * num_agents + other_id);
            const float other_I = msg.getVariable<float>("infectious");
            float other_pop = FLAMEGPU->environment.getProperty<float>("population_array", other_id);
            if (other_pop < 1.0f) {
                other_pop = 1.0f;
            }
            travel_force += prox * (other_I / other_pop);
        }
        travel_force *= beta_travel;

        float new_exposed = (local_force + travel_force) * S * dt;
        if (new_exposed > S) {
            new_exposed = S;
        }

        float new_infected = sigma * E * dt;
        if (new_infected > E) {
            new_infected = E;
        }

        float total_progress = gamma * I * dt;
        if (total_progress > I) {
            total_progress = I;
        }
        float deaths = total_progress * mortality;
        float recoveries = total_progress - deaths;

        S -= new_exposed;
        E += new_exposed - new_infected;
        I += new_infected - recoveries - deaths;
        R += recoveries;
        D += deaths;

        if (S < 0.0f) {
            S = 0.0f;
        }
        if (E < 0.0f) {
            E = 0.0f;
        }
        if (I < 0.0f) {
            I = 0.0f;
        }
        if (R < 0.0f) {
            R = 0.0f;
        }

        FLAMEGPU->setVariable<float>("S", S);
        FLAMEGPU->setVariable<float>("E", E);
        FLAMEGPU->setVariable<float>("I", I);
        FLAMEGPU->setVariable<float>("R", R);
        FLAMEGPU->setVariable<float>("D", D);

        return flamegpu::ALIVE;
    }
"""

# ----- funciones de agente -----
output_func = country_agent.newRTCFunction("output_infectious", output_infectious)
output_func.setMessageOutput("InfectiousMsg")
update_func = country_agent.newRTCFunction("update_state", update_state)
update_func.setMessageInput("InfectiousMsg")
update_func.dependsOn(output_func)
model.addExecutionRoot(output_func)

# ======================================================
# Environment
# ======================================================
proximity_list = proximity_matrix.flatten().astype(np.float32).tolist()
population_list = population.astype(np.float32).tolist()

env = model.Environment()
env.newPropertyUInt("num_agents", num_countries)
env.newPropertyArrayFloat("proximity_matrix", proximity_list)
env.newPropertyArrayFloat("population_array", population_list)
env.newPropertyFloat("beta_local", float(params["beta_local"]))
env.newPropertyFloat("beta_travel", float(params["beta_travel"]))
env.newPropertyFloat("sigma", float(params["sigma"]))
env.newPropertyFloat("gamma", float(params["gamma"]))
env.newPropertyFloat("mortality", float(params["mortality"]))
env.newPropertyFloat("dt", float(params["dt"]))

print("Modelo definido")


Modelo definido


In [26]:
# Construimos el vector inicial de agentes
init_population = fg.AgentVector(country_agent, num_countries)

for idx, row in state_df.iterrows():
    agent = init_population[idx]
    agent.setVariableUInt("id", idx)
    agent.setVariableFloat("population", row["Population"])
    agent.setVariableFloat("S", row["S0"])
    agent.setVariableFloat("E", row["E0"])
    agent.setVariableFloat("I", row["I0"])
    agent.setVariableFloat("R", row["R0"])
    agent.setVariableFloat("D", row["D0"])
    agent.setVariableFloat("lat", row["Latitud_promedio"])
    agent.setVariableFloat("lon", row["Longitud_promedio"])

print("Agentes inicializados")


Agentes inicializados


In [27]:
# Configuración de la simulación
simulation = fg.CUDASimulation(model)
simulation.SimulationConfig().random_seed = 42

simulation.setEnvironmentPropertyUInt("num_agents", num_countries)
simulation.setEnvironmentPropertyFloat("beta_local", float(params["beta_local"]))
simulation.setEnvironmentPropertyFloat("beta_travel", float(params["beta_travel"]))
simulation.setEnvironmentPropertyFloat("sigma", float(params["sigma"]))
simulation.setEnvironmentPropertyFloat("gamma", float(params["gamma"]))
simulation.setEnvironmentPropertyFloat("mortality", float(params["mortality"]))
simulation.setEnvironmentPropertyFloat("dt", float(params["dt"]))
simulation.setEnvironmentPropertyArrayFloat("proximity_matrix", proximity_list)
simulation.setEnvironmentPropertyArrayFloat("population_array", population_list)

simulation.setPopulationData(init_population)

simulation.SimulationConfig().steps = 1
simulation.simulate()



FLAMEGPURuntimeException: (InvalidAgentFunc) D:\a\FLAMEGPU2\FLAMEGPU2\src\flamegpu\detail\JitifyCache.cu(479): Error loading agent function (or function condition) ('output_infectious'): function had link errors:
Linking failed: NVJITLINK_ERROR_PTX_COMPILE
Linker options: "-arch=sm_86 --generate-line-info"


In [ ]:
def snapshot(simulation_obj):
    agent_vec = fg.AgentVector(country_agent)
    simulation_obj.getPopulationData(agent_vec)
    records = []
    for agent in agent_vec:
        idx = agent.getVariableUInt("id")
        records.append({
            "id": int(idx),
            "country": id_to_country[idx],
            "S": agent.getVariableFloat("S"),
            "E": agent.getVariableFloat("E"),
            "I": agent.getVariableFloat("I"),
            "R": agent.getVariableFloat("R"),
            "D": agent.getVariableFloat("D")
        })
    return records

print("Función snapshot lista")


In [ ]:
results = []

# Estado inicial (step = 0)
initial_records = snapshot(simulation)
for rec in initial_records:
    rec["step"] = 0
    rec["date"] = pd.NaT
    results.append(rec)

for step_idx, current_date in enumerate(unique_dates, start=1):
    simulation.step()
    step_records = snapshot(simulation)
    for rec in step_records:
        rec["step"] = step_idx
        rec["date"] = current_date
        results.append(rec)

results_df = pd.DataFrame(results)
print(f"Registros en resultados: {len(results_df):,}")
results_df.head()


In [ ]:
global_series = (
    results_df.groupby("step")["I"].sum()
               .reset_index()
               .rename(columns={"I": "Infectious_global"})
)

plt.figure(figsize=(10, 4))
plt.plot(global_series["step"], global_series["Infectious_global"], label="Infectados globales")
plt.xlabel("Paso (días)")
plt.ylabel("Personas")
plt.title("Evolución global de infectados (modelo)")
plt.grid(alpha=0.3)
plt.legend()
plt.show()


In [ ]:
latest_results = results_df[results_df["step"] == results_df["step"].max()]
top_countries = latest_results.nlargest(10, "I")[
    ["country", "S", "E", "I", "R", "D"]
]
print("Top 10 países por infectados al final de la simulación:")
top_countries


In [ ]:
SIM_OUTPUT_PATH = "data/covid_simulation_results.csv"
results_df.to_csv(SIM_OUTPUT_PATH, index=False)
print(f"Resultados completos guardados en {SIM_OUTPUT_PATH}")


## Notas finales
- El modelo es simplificado: S/E/I/R agregados por país, sin demografía.
- Podés ajustar los parámetros en `params` antes de re-ejecutar.
- La matriz de proximidades puede reemplazarse por valores custom (por ejemplo, del CSV de aristas) si querés experimentar.
- El archivo `data/covid_simulation_results.csv` guarda la evolución país-día de los estados.

Esto debería darte una base clara para continuar con experimentos más avanzados.


## Notas finales
- El modelo es simplificado: S/E/I/R agregados por país, sin demografía.
- Podés ajustar los parámetros en `params` antes de re-ejecutar.
- La matriz de proximidades puede reemplazarse por valores custom (por ejemplo, del CSV de aristas) si querés experimentar.
- El archivo `data/covid_simulation_results.csv` guarda la evolución país-día de los estados.

Esto debería darte una base clara para continuar con experimentos más avanzados.
